In [18]:
import numpy as np
import pandas as pd
import output_processing as op
from prompts import candidates

# GPT-4 output

def read_and_organize_model_results(model_name):
    model_results = pd.read_csv(f"runs/runs-42_07_16/{model_name}-results.csv")
    print(np.sum(model_results == 0.0))
    probs_columns = [candidate + "_probs" for candidate in candidates]
    # option 1
    # model_results.replace([0.0], -np.finfo(float).max, inplace=True)
    # option 2, set others to prob 0
    model_results = op.organize_distribution(model_results)
    def clean_other_probs_sure_answer():
        mask = model_results == 0.0
        # We only want the non-columns
        model_results.loc[mask]
        return model_results
    clean_other_probs_sure_answer()
    # for candidate in candidates: 
    #     probs_columns = [candidate + "_probs" for candidate in candidates]

    # print("Option 1:", np.sum((model_results["Yes_probs"] + model_results["No_probs"]) > 1))
    model_results["model_name"] = model_name.split("/")[-1]
    model_results.loc[model_results["Covered"] == True, "Judgment"] = "Covered"
    model_results.loc[model_results["Covered"] == True, "Judgment_prob"] = model_results["Covered_prob"]
    model_results.loc[model_results["NotCovered"] == True, "Judgment"] = "NotCovered"
    model_results.loc[model_results["NotCovered"] == True, "Judgment_prob"] = model_results["NotCovered_prob"]
    return model_results

model_results = read_and_organize_model_results("openai-gpt-4")

In [33]:
model_results.sample()

,title,prompt_type,prompt,version,output,output_text,YES_probs,Yes_probs,yes_probs,NO_probs,...,Aff_prob,UnAff_prob,Covered_prob,NotCovered_prob,Covered,NotCovered,entropy,model_name,Judgment,Judgment_prob
734,Hot Work II,disagreement,Martha has an insurance policy that covers acc...,controversial,No,No,0.00001,0.303124,0.000026,0.000007,...,0.696874,0.303124,0.696874,0.303124,True,False,0.613515,openai-gpt-4,Covered,0.696874


## OpenAI responses have > 1 total probability
https://cookbook.openai.com/examples/using_logprobs

OpenAI APIs provide logprob value 0.0 to indicate a token probability of 1
However, it assigns non-zero probability to tokens leading to a probability estimation > 1 over the vocabulary

There are two options to handle this
1. Assign the smallest non-zero negative logprob (`-MIN_FLOAT`)
2. Set others to 0
3. Normalize the probabilities over the top_20

## Does option 1, still lead to total probs > 1 : Yes for 200 cases

In [ ]:
# What about option 2